<a href="https://colab.research.google.com/github/Putradhika23/Forecasting_emergency_department/blob/main/Holt_Fix.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np 
import matplotlib.pyplot as plt
import plotly.express as px
import pandas as pd

from pandas import read_csv
from statsmodels.tsa.api import Holt
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_squared_error
from math import sqrt

#supress warning
import warnings
warnings.filterwarnings("ignore")

from google.colab import drive
drive.mount('/content/drive')

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


Mounted at /content/drive


In [ ]:
methods = ['Holt Alpha1 Beta1', 'Holt Alpha2 Beta2','Holt Alpha3 Beta3', 'Holt Alpha4 Beta4']
quality_metrics = ['MAE', 'MSE', 'RMSE', 'MAPE']
error_df = pd.DataFrame(columns = quality_metrics, index = methods)

def calculate_errors(df, method, test_data, forecast_data):
    residuals = test_data - forecast_data
    mae = np.mean(abs(residuals))
    mse = mean_squared_error(test_data, forecast_data)
    mape = round(np.mean(abs(residuals/test_data)),4)
    rmse = sqrt(mse)

    df.loc[method] = [mae, mse, rmse, mape]

In [ ]:
df = read_csv('/content/drive/My Drive/Colab Notebooks/dataset/data_arrival_igd.csv',sep=',')

df.head()

,DATE,ARRIVAL
0,2020-03-01,72
1,2020-03-02,88
2,2020-03-03,75
3,2020-03-04,81
4,2020-03-05,75


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 731 entries, 0 to 730
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   DATE     731 non-null    object
 1   ARRIVAL  731 non-null    int64 
dtypes: int64(1), object(1)
memory usage: 11.5+ KB


In [ ]:
fcast_horizon = 146

Split dataset

In [ ]:
splitter = len(df)- fcast_horizon

# Get the training data
training_data = df.iloc[:splitter,:]
display(training_data)

# Get the test data
test_data = df.iloc[splitter:,:]
display(test_data)

,DATE,ARRIVAL
0,2020-03-01,72
1,2020-03-02,88
2,2020-03-03,75
3,2020-03-04,81
4,2020-03-05,75
...,...,...
580,2021-10-01,48
581,2021-10-02,56
582,2021-10-03,56
583,2021-10-04,59


,DATE,ARRIVAL
585,2021-10-06,47
586,2021-10-07,46
587,2021-10-08,68
588,2021-10-09,69
589,2021-10-10,56
...,...,...
726,2022-02-24,65
727,2022-02-25,90
728,2022-02-26,72
729,2022-02-27,77


define parameter

In [ ]:
# Define alphas (level smoothing constants) and betas (trend smoothing constants)
alpha1 = 0.1
beta1 = 0.5

alpha2 = 0.2
beta2 = 0.8

alpha3 = 0.2
beta3 = 0.2

alpha4 = 0.3
beta4 = 0.3

build model

In [ ]:
# Build the Holt Model fpr alpha1 and beta1
fit1 = Holt(training_data.ARRIVAL).fit(smoothing_level = alpha1, smoothing_slope = beta1)
fit1.fittedvalues

0       88.000000
1      101.600000
2      114.760000
3      123.316000
4      129.500600
          ...    
580     61.437924
581     60.446709
582     60.132280
583     59.642680
584     59.469907
Length: 585, dtype: float64

In [ ]:
# Build the Holt Model fpr alpha2 and beta2
fit2 = Holt(training_data.ARRIVAL).fit(smoothing_level = alpha2, smoothing_slope = beta2)
fit2.fittedvalues

0       88.000000
1       98.240000
2      107.993600
3      107.917504
4      104.749827
          ...    
580     63.323281
581     57.515882
582     54.227423
583     51.880267
584     51.741700
Length: 585, dtype: float64

In [ ]:
# Build the Holt Model fpr alpha3 and beta3
fit3 = Holt(training_data.ARRIVAL).fit(smoothing_level = alpha3, smoothing_slope = beta3)
fit3.fittedvalues

0      84.243658
1      82.543663
2      84.601920
3      83.264449
4      83.303894
         ...    
580    61.622400
581    59.009560
582    58.398905
583    57.814425
584    57.994264
Length: 585, dtype: float64

In [ ]:
# Build the Holt Model fpr alpha4 and beta4
fit4 = Holt(training_data.ARRIVAL).fit(smoothing_level = alpha4, smoothing_slope = beta4)
fit4.fittedvalues

0      77.557361
1      77.852420
2      83.772244
3      83.226618
4      84.444285
         ...    
580    61.741236
581    56.752831
582    55.593192
583    54.818058
584    55.551839
Length: 585, dtype: float64

Forecast future

In [ ]:
fcast1 =  fit1.forecast(fcast_horizon)
fcast1

585    59.890915
586    59.958914
587    60.026912
588    60.094911
589    60.162910
         ...    
726    69.478751
727    69.546750
728    69.614749
729    69.682748
730    69.750746
Length: 146, dtype: float64

In [ ]:
fcast2 =  fit2.forecast(fcast_horizon)
fcast2

585    54.232174
586    54.470989
587    54.709803
588    54.948617
589    55.187432
         ...    
726    87.904995
727    88.143809
728    88.382623
729    88.621438
730    88.860252
Length: 146, dtype: float64

In [ ]:
fcast3 =  fit3.forecast(fcast_horizon)
fcast3

585    59.138365
586    59.281319
587    59.424272
588    59.567226
589    59.710179
         ...    
726    79.294817
727    79.437770
728    79.580724
729    79.723678
730    79.866631
Length: 146, dtype: float64

In [ ]:
fcast4 =  fit4.forecast(fcast_horizon)
fcast4

585    57.935820
586    58.085353
587    58.234885
588    58.384418
589    58.533951
         ...    
726    79.019933
727    79.169465
728    79.318998
729    79.468531
730    79.618063
Length: 146, dtype: float64

Plot forecast

In [ ]:
df['fcast1'] = fcast1

df
# Plot using plotly
fig = px.line(df, x='DATE', y=['ARRIVAL','Holt Exp'] ,width=900, height=400)
fig.show()

In [ ]:
df['fcast1'] = fcast1
df['fcast2'] = fcast2
df['fcast3'] = fcast3
df['fcast4'] = fcast4
df
# Plot using plotly
fig = px.line(df, x='DATE', y=['ARRIVAL','fcast1','fcast2','fcast3','fcast4'])
fig.show()

Calculate Error

In [ ]:
calculate_errors(error_df, 'Holt Alpha1 Beta1', test_data.ARRIVAL, fcast1)
error_df

,MAE,MSE,RMSE,MAPE
Holt Alpha1 Beta1,8.169746,103.672844,10.181986,0.1119
Holt Alpha2 Beta2,7.672142,90.546187,9.515576,0.1123
Holt Alpha3 Beta3,6.915949,69.555871,8.340016,0.1004
Holt Alpha4 Beta4,6.981014,70.923921,8.421634,0.1002


In [ ]:
calculate_errors(error_df, 'Holt Alpha2 Beta2', test_data.ARRIVAL, fcast2)
error_df

,MAE,MSE,RMSE,MAPE
Holt Alpha1 Beta1,16.152023,365.678058,19.122711,0.2162
Holt Alpha2 Beta2,31.57925,1314.659764,36.258237,0.4287
Holt Alpha3 Beta3,12.467328,224.96458,14.998819,0.1857
Holt Alpha4 Beta4,6.915949,69.555871,8.340016,0.1004


In [ ]:
calculate_errors(error_df, 'Holt Alpha3 Beta3', test_data.ARRIVAL, fcast3)
error_df

,MAE,MSE,RMSE,MAPE
Holt Alpha1 Beta1,16.152023,365.678058,19.122711,0.2162
Holt Alpha2 Beta2,31.57925,1314.659764,36.258237,0.4287
Holt Alpha3 Beta3,36.084231,1693.867242,41.156618,0.4914
Holt Alpha4 Beta4,6.915949,69.555871,8.340016,0.1004


In [ ]:
calculate_errors(error_df, 'Holt Alpha4 Beta4', test_data.ARRIVAL, fcast4)
error_df

,MAE,MSE,RMSE,MAPE
Holt Alpha1 Beta1,16.152023,365.678058,19.122711,0.2162
Holt Alpha2 Beta2,31.57925,1314.659764,36.258237,0.4287
Holt Alpha3 Beta3,36.084231,1693.867242,41.156618,0.4914
Holt Alpha4 Beta4,29.550238,1130.132115,33.617438,0.4012


In [ ]:
def double_exponential_smoothing(series, alpha, beta):

    result = [series[0]]
    for n in range(1, len(series)+1):
        if n == 1:
            level, trend = series[0], series[1] - series[0]
        if n >= len(series): # forecasting
            value = result[-1]
        else:
            value = series[n]
        last_level, level = level, alpha * value + (1 - alpha) * (level + trend)
        trend = beta * (level - last_level) + (1 - beta) * trend
        result.append(level + trend)
    return result

def plot_double_exponential_smoothing(series, alphas, betas):
     
    plt.figure(figsize=(17, 8))
    for alpha in alphas:
        for beta in betas:
            plt.plot(double_exponential_smoothing(series, alpha, beta), label="Alpha {}, beta {}".format(alpha, beta))
    plt.plot(series.values, label = "Actual")
    plt.legend(loc="best")
    plt.axis('tight')
    plt.title("Double Exponential Smoothing")
    plt.grid(True)
    
plot_double_exponential_smoothing(df.CLOSE, alphas=[0.9, 0.02], betas=[0.9, 0.02])

AttributeError: ignored